In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
df_test = pd.read_csv('../input/commonlitreadabilityprize/test.csv')
df_sample = pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')

In [ ]:
df_train.sample(frac = 0.10)

In [ ]:
df_test.sample(frac = 0.5)

In [ ]:
df_sample.sample(frac= 0.5)

In [ ]:
df_train = df_train.replace(np.nan, '', regex=True)

In [ ]:
df_train['excerpt_1'] = df_train.excerpt+' '+df_train.url_legal+' '+df_train.license

In [ ]:
df_train.head()

In [ ]:
df_train.excerpt_1 = df_train.excerpt_1.str.lower()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
vectorizer = TfidfVectorizer(analyzer='char',ngram_range=(1,6))
X = vectorizer.fit_transform(df_train.excerpt_1,)
X.shape

In [ ]:
from sklearn.svm import LinearSVR
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

In [ ]:
model = LinearSVR(random_state=1994)

In [ ]:
scores = cross_val_score(model, X, df_train.target, cv=5,scoring='neg_root_mean_squared_error')
scores *=-1
scores.mean()

In [ ]:
model.fit(X,df_train.target)

In [ ]:
df_test = pd.read_csv('../input/commonlitreadabilityprize/test.csv',index_col='id')
df_test = df_test.replace(np.nan, '', regex=True)
df_test['excerpt_1'] = df_test.excerpt+' '+df_test.url_legal+' '+df_test.license
df_test.excerpt_1 = df_test.excerpt_1.str.lower()
x = vectorizer.transform(df_test.excerpt_1)
p = model.predict(x)
df_test['target'] = p
df_test[['target']].to_csv('submission.csv')